# PFT_GEM Tutorial: Geometric Expansion Model for Tumor Displacement

This notebook demonstrates how to use PFT_GEM (Posterior Fossa Tumor - Geometric Expansion Model) to simulate tumor-induced brain displacement using a computationally efficient geometric expansion approach.

## Overview

PFT_GEM provides a simplified alternative to finite element methods for modeling how brain tumors displace surrounding tissue. The model is particularly useful for:

- Rapid estimation of displacement fields
- Visualization of tumor mass effect
- Integration with diffusion MRI for biophysical constraints
- Research on posterior fossa tumors (cerebellum, brainstem)

## Contents

1. [Setup and Imports](#1.-Setup-and-Imports)
2. [Loading SUIT Template Data](#2.-Loading-SUIT-Template-Data)
3. [Loading DTI Reference Data](#3.-Loading-DTI-Reference-Data)
4. [Creating a Tumor Model](#4.-Creating-a-Tumor-Model)
5. [Computing Displacement Fields](#5.-Computing-Displacement-Fields)
6. [Applying DTI Constraints](#6.-Applying-DTI-Constraints)
7. [Generating Synthetic MRI Output](#7.-Generating-Synthetic-MRI-Output)
8. [Saving and Loading Results](#8.-Saving-and-Loading-Results)
9. [Advanced: Analyzing Strain and Volume Changes](#9.-Advanced:-Analyzing-Strain-and-Volume-Changes)

## 1. Setup and Imports

In [ ]:
# Install PFT_GEM if not already installed
# !pip install -e ..

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

# Import PFT_GEM modules
from pft_gem import GeometricExpansionModel, DisplacementField, BiophysicalConstraints
from pft_gem.core.geometric_model import TumorParameters, ModelParameters
from pft_gem.core.displacement import FieldMetadata
from pft_gem.core.constraints import DTIData, create_synthetic_dti
from pft_gem.io import (
    SUITTemplateLoader, 
    TemplateData,
    save_displacement_field,
    load_displacement_field,
    generate_synthetic_mri_output,
    save_synthetic_output,
    save_displacement_as_warp,
)
from pft_gem.visualization import (
    plot_displacement_field,
    plot_displacement_magnitude,
    plot_vector_field,
    plot_jacobian,
    plot_tumor_displacement_profile,
    plot_slice_comparison,
    VisualizationConfig
)
from pft_gem.utils import create_spherical_tumor_mask, create_ellipsoidal_tumor_mask

# Set paths to data directory
DATA_DIR = Path('../data')
SUIT_DIR = DATA_DIR / 'suit_template'
DTI_DIR = DATA_DIR / 'HCP1065_DTI'

print("PFT_GEM modules loaded successfully!")
print(f"Data directory: {DATA_DIR.absolute()}")

## 2. Loading SUIT Template Data

The SUIT (Spatially Unbiased Infratentorial Template) provides high-resolution reference data for the cerebellum and brainstem. This is the ideal template for posterior fossa tumor modeling.

In [ ]:
# Load SUIT template data
suit_loader = SUITTemplateLoader(SUIT_DIR)

if suit_loader.is_available():
    template_data = suit_loader.load_template()
    print("SUIT template loaded successfully!")
    print(f"Template shape: {template_data.template.shape}")
    print(f"Voxel size: {template_data.voxel_size} mm")
    print(f"Number of atlas regions: {len(template_data.labels) - 1}")  # Exclude background
else:
    print("SUIT template not found. Creating synthetic template for demonstration.")
    template_data = SUITTemplateLoader.create_synthetic_template(
        shape=(128, 128, 64),
        voxel_size=(1.0, 1.0, 1.0)
    )

In [ ]:
# Visualize the SUIT template in three views
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Get center slices
shape = template_data.template.shape
cx, cy, cz = shape[0]//2, shape[1]//2, shape[2]//2

# T1 Template
axes[0, 0].imshow(template_data.template[cx, :, :].T, origin='lower', cmap='gray')
axes[0, 0].set_title(f'T1 Template - Sagittal (x={cx})')
axes[0, 1].imshow(template_data.template[:, cy, :].T, origin='lower', cmap='gray')
axes[0, 1].set_title(f'T1 Template - Coronal (y={cy})')
axes[0, 2].imshow(template_data.template[:, :, cz].T, origin='lower', cmap='gray')
axes[0, 2].set_title(f'T1 Template - Axial (z={cz})')

# Atlas parcellation
if template_data.atlas is not None:
    axes[1, 0].imshow(template_data.atlas[cx, :, :].T, origin='lower', cmap='nipy_spectral')
    axes[1, 0].set_title('Atlas - Sagittal')
    axes[1, 1].imshow(template_data.atlas[:, cy, :].T, origin='lower', cmap='nipy_spectral')
    axes[1, 1].set_title('Atlas - Coronal')
    axes[1, 2].imshow(template_data.atlas[:, :, cz].T, origin='lower', cmap='nipy_spectral')
    axes[1, 2].set_title('Atlas - Axial')

plt.tight_layout()
plt.suptitle('SUIT Template Data', fontsize=14, y=1.02)
plt.show()

In [ ]:
# Display atlas region labels
print("SUIT Atlas Regions:")
print("="*50)
for idx, name in sorted(template_data.labels.items()):
    if idx > 0:  # Skip background
        print(f"  {idx:2d}: {name}")

In [ ]:
# Get specific region masks
left_hemisphere = suit_loader.get_left_hemisphere_mask()
right_hemisphere = suit_loader.get_right_hemisphere_mask()
vermis = suit_loader.get_vermis_mask()
deep_nuclei = suit_loader.get_deep_nuclei_mask()

print("Region volumes (in voxels):")
if left_hemisphere is not None:
    print(f"  Left hemisphere: {np.sum(left_hemisphere)}")
if right_hemisphere is not None:
    print(f"  Right hemisphere: {np.sum(right_hemisphere)}")
if vermis is not None:
    print(f"  Vermis: {np.sum(vermis)}")
if deep_nuclei is not None:
    print(f"  Deep nuclei: {np.sum(deep_nuclei)}")

## 3. Loading DTI Reference Data

The HCP1065 DTI atlas provides population-averaged diffusion tensor imaging data that can be used to inform the biophysical constraints of our displacement model.

In [ ]:
import nibabel as nib

# Load DTI data
fa_path = DTI_DIR / 'FSL_HCP1065_FA_1mm.nii.gz'
v1_path = DTI_DIR / 'FSL_HCP1065_V1_1mm.nii.gz'

if fa_path.exists() and v1_path.exists():
    fa_img = nib.load(str(fa_path))
    v1_img = nib.load(str(v1_path))
    
    fa_full = fa_img.get_fdata()
    v1_full = v1_img.get_fdata()
    
    print("DTI data loaded successfully!")
    print(f"FA map shape: {fa_full.shape}")
    print(f"V1 (principal eigenvector) shape: {v1_full.shape}")
    print(f"FA range: [{fa_full.min():.3f}, {fa_full.max():.3f}]")
    print(f"DTI voxel size: {fa_img.header.get_zooms()[:3]} mm")
else:
    print("DTI data not found. Will use synthetic DTI data.")
    fa_full = None
    v1_full = None

In [ ]:
# Visualize DTI data (full MNI space view)
if fa_full is not None:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # Get center slices in MNI space
    mni_cx, mni_cy, mni_cz = fa_full.shape[0]//2, fa_full.shape[1]//2, fa_full.shape[2]//2
    
    # FA maps
    axes[0, 0].imshow(fa_full[mni_cx, :, :].T, origin='lower', cmap='hot', vmin=0, vmax=1)
    axes[0, 0].set_title(f'FA - Sagittal (x={mni_cx})')
    axes[0, 1].imshow(fa_full[:, mni_cy, :].T, origin='lower', cmap='hot', vmin=0, vmax=1)
    axes[0, 1].set_title(f'FA - Coronal (y={mni_cy})')
    im = axes[0, 2].imshow(fa_full[:, :, mni_cz].T, origin='lower', cmap='hot', vmin=0, vmax=1)
    axes[0, 2].set_title(f'FA - Axial (z={mni_cz})')
    plt.colorbar(im, ax=axes[0, 2], label='FA')
    
    # Principal eigenvector (color-coded RGB)
    v1_rgb_sag = np.abs(v1_full[mni_cx, :, :, :]) 
    v1_rgb_cor = np.abs(v1_full[:, mni_cy, :, :])
    v1_rgb_ax = np.abs(v1_full[:, :, mni_cz, :])
    
    # Normalize
    v1_rgb_sag = v1_rgb_sag / (v1_rgb_sag.max() + 1e-10)
    v1_rgb_cor = v1_rgb_cor / (v1_rgb_cor.max() + 1e-10)
    v1_rgb_ax = v1_rgb_ax / (v1_rgb_ax.max() + 1e-10)
    
    axes[1, 0].imshow(np.transpose(v1_rgb_sag, (1, 0, 2)), origin='lower')
    axes[1, 0].set_title('V1 Direction - Sagittal (RGB=XYZ)')
    axes[1, 1].imshow(np.transpose(v1_rgb_cor, (1, 0, 2)), origin='lower')
    axes[1, 1].set_title('V1 Direction - Coronal')
    axes[1, 2].imshow(np.transpose(v1_rgb_ax, (1, 0, 2)), origin='lower')
    axes[1, 2].set_title('V1 Direction - Axial')
    
    plt.tight_layout()
    plt.suptitle('HCP1065 DTI Reference Data (MNI Space)', fontsize=14, y=1.02)
    plt.show()

## 4. Creating a Tumor Model

Now we'll define a tumor in the SUIT template space. We'll place it in a typical location for a posterior fossa tumor - in the cerebellar hemisphere.

In [ ]:
# Get template properties
grid_shape = template_data.template.shape
voxel_size = template_data.voxel_size

# Define tumor location in the right cerebellar hemisphere
# (approximately in the Crus I/II region)
tumor_center = (
    grid_shape[0] * 0.35,  # Slightly right of midline
    grid_shape[1] * 0.55,  # Posterior
    grid_shape[2] * 0.45   # Mid-height
)
tumor_radius = 12.0  # 12mm radius (24mm diameter tumor)

print(f"Grid shape: {grid_shape}")
print(f"Voxel size: {voxel_size} mm")
print(f"Tumor center (voxels): ({tumor_center[0]:.1f}, {tumor_center[1]:.1f}, {tumor_center[2]:.1f})")
print(f"Tumor radius: {tumor_radius} mm")

# Define tumor parameters
tumor_params = TumorParameters(
    center=tumor_center,
    radius=tumor_radius,
    shape="spherical"
)

# Define model parameters
model_params = ModelParameters(
    decay_exponent=2.0,        # How quickly displacement decays with distance
    boundary_stiffness=0.8,    # Stiffness factor at anatomical boundaries
    csf_damping=0.1,           # Damping in CSF regions
    max_displacement=15.0,     # Maximum allowed displacement (mm)
    tissue_modulation=True,    # Apply tissue-specific modulation
    use_dti_constraints=True,  # Use DTI-derived constraints
    smoothing_sigma=1.0        # Gaussian smoothing (mm)
)

In [ ]:
# Create the geometric expansion model
model = GeometricExpansionModel(
    tumor_params=tumor_params,
    model_params=model_params,
    grid_shape=grid_shape,
    voxel_size=voxel_size
)

# Set boundary mask from SUIT cerebellum mask
if template_data.mask is not None:
    # Resample mask if needed
    from scipy.ndimage import zoom
    if template_data.mask.shape != grid_shape:
        scale = [g/m for g, m in zip(grid_shape, template_data.mask.shape)]
        mask_resampled = zoom(template_data.mask, scale, order=0)
        model.set_boundary_mask(mask_resampled)
    else:
        model.set_boundary_mask(template_data.mask)

print("Model created!")

In [ ]:
# Visualize tumor location on the template
tumor_mask = create_spherical_tumor_mask(
    grid_shape, 
    tumor_center, 
    tumor_radius
)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

slice_x = int(tumor_center[0])
slice_y = int(tumor_center[1])
slice_z = int(tumor_center[2])

# Sagittal
axes[0].imshow(template_data.template[slice_x, :, :].T, origin='lower', cmap='gray')
axes[0].contour(tumor_mask[slice_x, :, :].T, levels=[0.5], colors='red', linewidths=2)
axes[0].set_title(f'Sagittal (x={slice_x})')

# Coronal
axes[1].imshow(template_data.template[:, slice_y, :].T, origin='lower', cmap='gray')
axes[1].contour(tumor_mask[:, slice_y, :].T, levels=[0.5], colors='red', linewidths=2)
axes[1].set_title(f'Coronal (y={slice_y})')

# Axial
axes[2].imshow(template_data.template[:, :, slice_z].T, origin='lower', cmap='gray')
axes[2].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='red', linewidths=2)
axes[2].set_title(f'Axial (z={slice_z})')

plt.suptitle('Tumor Location (red outline) on SUIT Template', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Computing Displacement Fields

The geometric expansion model computes displacement based on:

$$u(r) = u_0 \cdot \left(\frac{R}{r}\right)^\alpha \cdot f(\text{tissue}) \cdot g(\text{boundaries})$$

Where:
- $u_0$: displacement at tumor boundary
- $R$: tumor radius
- $r$: distance from tumor center
- $\alpha$: decay exponent

In [ ]:
# Compute displacement field for a 5mm tumor expansion
tumor_expansion = 5.0  # mm
displacement = model.compute_displacement_field(tumor_expansion=tumor_expansion)

print(f"Displacement field shape: {displacement.shape}")
print(f"Components: (x, y, z) displacement at each voxel")

# Compute magnitude
magnitude = np.linalg.norm(displacement, axis=-1)
print(f"\nDisplacement statistics:")
print(f"  Max displacement: {magnitude.max():.2f} mm")
print(f"  Mean displacement: {magnitude.mean():.2f} mm")
print(f"  Non-zero voxels: {np.sum(magnitude > 0.01)}/{magnitude.size}")

In [ ]:
# Visualize the displacement magnitude in three views
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Sagittal view
im0 = axes[0].imshow(magnitude[slice_x, :, :].T, origin='lower', cmap='hot')
axes[0].contour(tumor_mask[slice_x, :, :].T, levels=[0.5], colors='cyan', linewidths=2)
axes[0].set_title(f'Sagittal (x={slice_x})')
axes[0].set_xlabel('Y (voxels)')
axes[0].set_ylabel('Z (voxels)')

# Coronal view
im1 = axes[1].imshow(magnitude[:, slice_y, :].T, origin='lower', cmap='hot')
axes[1].contour(tumor_mask[:, slice_y, :].T, levels=[0.5], colors='cyan', linewidths=2)
axes[1].set_title(f'Coronal (y={slice_y})')
axes[1].set_xlabel('X (voxels)')
axes[1].set_ylabel('Z (voxels)')

# Axial view
im2 = axes[2].imshow(magnitude[:, :, slice_z].T, origin='lower', cmap='hot')
axes[2].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='cyan', linewidths=2)
axes[2].set_title(f'Axial (z={slice_z})')
axes[2].set_xlabel('X (voxels)')
axes[2].set_ylabel('Y (voxels)')

fig.colorbar(im2, ax=axes, label='Displacement (mm)', shrink=0.8)
fig.suptitle('Displacement Magnitude (tumor boundary shown in cyan)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Plot displacement vectors
fig = plot_vector_field(
    displacement,
    slice_idx=slice_z,
    axis=2,
    background=template_data.template,
    config=VisualizationConfig(vector_density=6, vector_scale=2.0)
)
plt.title('Displacement Vector Field (axial slice)')
plt.show()

In [ ]:
# Plot displacement profile as a function of distance from tumor
fig = plot_tumor_displacement_profile(
    displacement,
    tumor_center=(int(tumor_center[0]), int(tumor_center[1]), int(tumor_center[2])),
    directions=["x", "y", "z", "radial"],
    max_distance=50
)
plt.axvline(x=tumor_radius, color='gray', linestyle='--', label='Tumor boundary')
plt.legend()
plt.show()

## 6. Applying DTI Constraints

DTI data provides information about tissue microstructure that can modulate the displacement field:

- **Fractional Anisotropy (FA)**: Higher FA indicates more organized white matter (stiffer)
- **Mean Diffusivity (MD)**: Lower MD indicates denser tissue (stiffer)

In [ ]:
# Create DTI data for the model (resampled to SUIT space or synthetic)
# Note: The HCP DTI data is in MNI space, which would need transformation to SUIT space
# For demonstration, we'll create synthetic DTI in SUIT space

dti_data = create_synthetic_dti(
    shape=grid_shape,
    tumor_center=(int(tumor_center[0]), int(tumor_center[1]), int(tumor_center[2])),
    tumor_radius=tumor_radius
)

print(f"DTI data for model:")
print(f"  FA range: [{dti_data.fa.min():.3f}, {dti_data.fa.max():.3f}]")
print(f"  MD range: [{dti_data.md.min()*1000:.2f}, {dti_data.md.max()*1000:.2f}] x 10^-3 mm²/s")

In [ ]:
# Create biophysical constraints from DTI
constraints = BiophysicalConstraints(dti_data)

# Compute stiffness map from DTI
stiffness = constraints.compute_stiffness_map(normalize=True)

# Visualize stiffness map
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

im0 = axes[0].imshow(dti_data.fa[:, :, slice_z].T, origin='lower', cmap='viridis', vmin=0, vmax=1)
axes[0].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='red', linewidths=2)
axes[0].set_title('Fractional Anisotropy (FA)')
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(dti_data.md[:, :, slice_z].T * 1000, origin='lower', cmap='plasma')
axes[1].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='red', linewidths=2)
axes[1].set_title('Mean Diffusivity (MD × 10³)')
plt.colorbar(im1, ax=axes[1], label='mm²/s × 10³')

im2 = axes[2].imshow(stiffness[:, :, slice_z].T, origin='lower', cmap='coolwarm')
axes[2].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='black', linewidths=2)
axes[2].set_title('Derived Tissue Stiffness')
plt.colorbar(im2, ax=axes[2], label='Relative stiffness')

plt.tight_layout()
plt.show()

In [ ]:
# Apply DTI constraints and recompute displacement
model.set_dti_constraints(dti_data.fa, dti_data.md)
displacement_dti = model.compute_displacement_field(tumor_expansion=tumor_expansion)
magnitude_dti = np.linalg.norm(displacement_dti, axis=-1)

print(f"Without DTI constraints:")
print(f"  Max displacement: {magnitude.max():.2f} mm")
print(f"  Mean displacement: {magnitude.mean():.2f} mm")
print(f"\nWith DTI constraints:")
print(f"  Max displacement: {magnitude_dti.max():.2f} mm")
print(f"  Mean displacement: {magnitude_dti.mean():.2f} mm")

In [ ]:
# Compare displacement with and without DTI constraints
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

vmax = max(magnitude.max(), magnitude_dti.max())

# Without DTI
im0 = axes[0].imshow(magnitude[:, :, slice_z].T, origin='lower', cmap='hot', vmax=vmax)
axes[0].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='cyan', linewidths=2)
axes[0].set_title('Without DTI Constraints')
plt.colorbar(im0, ax=axes[0], label='mm')

# With DTI
im1 = axes[1].imshow(magnitude_dti[:, :, slice_z].T, origin='lower', cmap='hot', vmax=vmax)
axes[1].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='cyan', linewidths=2)
axes[1].set_title('With DTI Constraints')
plt.colorbar(im1, ax=axes[1], label='mm')

# Difference
diff = magnitude_dti - magnitude
vmax_diff = np.abs(diff).max()
im2 = axes[2].imshow(diff[:, :, slice_z].T, origin='lower', cmap='RdBu_r', 
                      vmin=-vmax_diff, vmax=vmax_diff)
axes[2].contour(tumor_mask[:, :, slice_z].T, levels=[0.5], colors='black', linewidths=2)
axes[2].set_title('Difference (DTI - baseline)')
plt.colorbar(im2, ax=axes[2], label='mm')

fig.suptitle('Effect of DTI Constraints on Displacement', fontsize=14)
plt.tight_layout()
plt.show()

## 7. Generating Synthetic MRI Output

The ultimate output of this pipeline is:
1. **Synthetic MRI data**: The SUIT template warped by the displacement field, showing what the brain would look like with the tumor present
2. **Displacement field as a transform**: A NIfTI warp file that can be applied to other images

These outputs can be used with standard neuroimaging tools (FSL, ANTs, etc.).

In [ ]:
# Generate synthetic MRI output using the convenience function
from pft_gem.io import generate_synthetic_mri_output, save_synthetic_output

# Generate the complete output
synthetic_output = generate_synthetic_mri_output(
    template_data=template_data,
    tumor_center=tumor_center,
    tumor_radius=tumor_radius,
    tumor_expansion=tumor_expansion,
    decay_exponent=2.0
)

print("Synthetic MRI output generated!")
print(f"  Displaced image shape: {synthetic_output.displaced_image.shape}")
print(f"  Displacement field shape: {synthetic_output.displacement_field.shape}")
print(f"  Tumor mask voxels: {np.sum(synthetic_output.tumor_mask)}")

In [ ]:
# Visualize the displaced template vs original
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Original template
axes[0, 0].imshow(synthetic_output.original_image[slice_x, :, :].T, origin='lower', cmap='gray')
axes[0, 0].set_title('Original - Sagittal')
axes[0, 1].imshow(synthetic_output.original_image[:, slice_y, :].T, origin='lower', cmap='gray')
axes[0, 1].set_title('Original - Coronal')
axes[0, 2].imshow(synthetic_output.original_image[:, :, slice_z].T, origin='lower', cmap='gray')
axes[0, 2].set_title('Original - Axial')

# Displaced template with tumor
axes[1, 0].imshow(synthetic_output.displaced_image[slice_x, :, :].T, origin='lower', cmap='gray')
axes[1, 0].contour(synthetic_output.tumor_mask[slice_x, :, :].T, levels=[0.5], colors='red', linewidths=2)
axes[1, 0].set_title('Displaced - Sagittal')

axes[1, 1].imshow(synthetic_output.displaced_image[:, slice_y, :].T, origin='lower', cmap='gray')
axes[1, 1].contour(synthetic_output.tumor_mask[:, slice_y, :].T, levels=[0.5], colors='red', linewidths=2)
axes[1, 1].set_title('Displaced - Coronal')

axes[1, 2].imshow(synthetic_output.displaced_image[:, :, slice_z].T, origin='lower', cmap='gray')
axes[1, 2].contour(synthetic_output.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='red', linewidths=2)
axes[1, 2].set_title('Displaced - Axial')

plt.suptitle('Original vs Displaced Template (tumor boundary in red)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Show the difference between original and displaced
fig = plot_slice_comparison(
    synthetic_output.original_image,
    synthetic_output.displaced_image,
    slice_idx=slice_z,
    axis=2,
    difference=True
)
fig.suptitle('Template Deformation by Tumor (Axial View)', fontsize=14)
plt.show()

## 8. Saving and Loading Results

Save all outputs to NIfTI files that can be used with standard neuroimaging tools.

In [ ]:
# Save all synthetic outputs
output_dir = Path('../output')
output_dir.mkdir(exist_ok=True)

saved_paths = save_synthetic_output(
    synthetic_output,
    output_dir,
    prefix="pft_gem_example"
)

print("Saved files:")
for name, path in saved_paths.items():
    print(f"  {name}: {path}")

In [ ]:
# The warp file can be used with other tools
print("\nThe displacement field (warp) can be applied to other images.")
print("\nFSL example:")
print(f"  applywarp -i input.nii.gz -r reference.nii.gz -w {saved_paths['warp']} -o output.nii.gz")
print("\nANTs example:")
print(f"  antsApplyTransforms -d 3 -i input.nii.gz -r reference.nii.gz -t {saved_paths['warp']} -o output.nii.gz")

In [ ]:
# Demonstrate applying the warp to the atlas
if template_data.atlas is not None:
    from pft_gem.io.synthetic_output import generate_displaced_template
    
    # Warp the atlas using nearest neighbor interpolation (to preserve labels)
    warped_atlas = generate_displaced_template(
        template_data.atlas.astype(np.float32),
        synthetic_output.displacement_field,
        voxel_size=synthetic_output.voxel_size,
        order=0  # Nearest neighbor for labels
    )
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(template_data.atlas[:, :, slice_z].T, origin='lower', cmap='nipy_spectral')
    axes[0].set_title('Original Atlas')
    
    axes[1].imshow(warped_atlas[:, :, slice_z].T, origin='lower', cmap='nipy_spectral')
    axes[1].contour(synthetic_output.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='white', linewidths=2)
    axes[1].set_title('Warped Atlas (with tumor)')
    
    # Show which regions are displaced
    diff_atlas = np.abs(warped_atlas - template_data.atlas)
    axes[2].imshow(diff_atlas[:, :, slice_z].T > 0, origin='lower', cmap='Reds')
    axes[2].contour(synthetic_output.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='cyan', linewidths=2)
    axes[2].set_title('Regions Affected by Displacement')
    
    plt.suptitle('Atlas Deformation', fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
# Export model configuration
import json

config = model.to_dict()
print("Model configuration:")
print(json.dumps(config, indent=2, default=str))

## 9. Advanced: Analyzing Strain and Volume Changes

The displacement field can be analyzed to understand local tissue deformation.

In [ ]:
# Compute Jacobian determinant (local volume change)
jacobian = model.compute_jacobian_determinant()

print(f"Jacobian statistics:")
print(f"  Min: {jacobian.min():.3f} (max compression)")
print(f"  Max: {jacobian.max():.3f} (max expansion)")
print(f"  Mean: {jacobian.mean():.3f}")
print(f"  Voxels with J > 1.1 (>10% expansion): {np.sum(jacobian > 1.1)}")
print(f"  Voxels with J < 0.9 (<10% compression): {np.sum(jacobian < 0.9)}")
print(f"  Voxels with J < 0 (folding): {np.sum(jacobian < 0)}")

In [ ]:
# Visualize Jacobian
fig = plot_jacobian(
    jacobian,
    slice_idx=slice_z,
    axis=2,
    symmetric_scale=True
)
plt.show()

In [ ]:
# Compute strain tensor
strain = model.compute_strain_field()
print(f"Strain tensor shape: {strain.shape}")

# Compute principal strains (eigenvalues)
principal_strains = np.linalg.eigvalsh(strain)
max_strain = np.max(np.abs(principal_strains), axis=-1)

print(f"Maximum principal strain: {max_strain.max():.4f}")

In [ ]:
# Visualize maximum principal strain
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, (ax, axis, name) in enumerate(zip(axes, [0, 1, 2], ['Sagittal', 'Coronal', 'Axial'])):
    if axis == 0:
        slice_data = max_strain[slice_x, :, :]
        tumor_slice = tumor_mask[slice_x, :, :]
    elif axis == 1:
        slice_data = max_strain[:, slice_y, :]
        tumor_slice = tumor_mask[:, slice_y, :]
    else:
        slice_data = max_strain[:, :, slice_z]
        tumor_slice = tumor_mask[:, :, slice_z]
    
    im = ax.imshow(slice_data.T, origin='lower', cmap='magma')
    ax.contour(tumor_slice.T, levels=[0.5], colors='cyan', linewidths=2)
    ax.set_title(f'{name}')
    plt.colorbar(im, ax=ax)

plt.suptitle('Maximum Principal Strain', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Get comprehensive field statistics
field = DisplacementField(
    displacement_dti,
    metadata=FieldMetadata(shape=grid_shape, voxel_size=voxel_size)
)

# Use cerebellum mask if available
mask = template_data.mask if template_data.mask is not None else None
if mask is not None and mask.shape != grid_shape:
    from scipy.ndimage import zoom
    scale = [g/m for g, m in zip(grid_shape, mask.shape)]
    mask = zoom(mask, scale, order=0)

stats = field.statistics(mask=mask)
print("Displacement field statistics (within cerebellum mask):")
for category, values in stats.items():
    print(f"\n{category.upper()}:")
    for key, val in values.items():
        print(f"  {key}: {val:.4f}")

## Summary

In this tutorial, we covered:

1. **Loading SUIT template data** for anatomically accurate cerebellum representation
2. **Loading DTI reference data** from the HCP1065 atlas
3. **Creating a geometric expansion model** with tumor parameters
4. **Computing displacement fields** showing tissue deformation
5. **Adding DTI-derived biophysical constraints** for more realistic results
6. **Generating synthetic MRI output** - displaced templates and warp fields
7. **Saving outputs** in NIfTI format compatible with standard tools
8. **Analyzing strain and volume changes** from the deformation

### Key Outputs

The pipeline produces:
- **Displaced template** (`*_displaced.nii.gz`): Synthetic MRI showing tumor effect
- **Displacement field** (`*_warp.nii.gz`): Transform applicable to other images
- **Inverse warp** (`*_warp_inverse.nii.gz`): For reverse transformation
- **Tumor mask** (`*_tumor_mask.nii.gz`): Binary tumor region

### Key Advantages of PFT_GEM

- **Computational efficiency**: Much faster than FEM approaches
- **Biophysical plausibility**: DTI constraints ensure realistic tissue behavior
- **Easy integration**: NIfTI format compatible with standard neuroimaging tools
- **Flexibility**: Supports spherical and ellipsoidal tumors, various parameter tuning

### Next Steps

- Use patient-specific DTI data for personalized modeling
- Apply warps to functional MRI or other modalities
- Explore parameter sensitivity for your specific application
- Compare results with FEM-based approaches (PFT_FEM)